# 75 pandas Exercises: Exercises 31 to 40

Exercises 31 to 40 from [here](https://www.machinelearningplus.com/python/101-pandas-exercises-python/). Each exercise includes the question, the input and the solution's code. Sometimes, alternative solutions and comments to better explain solutions/pandas functionality are offered.

Requirements: 
+ `pandas`
+ `numpy`

Happy Pandasing! 🐼

# Imports 

In [5]:
import numpy as np
import pandas as pd

---

## Exercises

### 🐼 Exercise 31